In [1]:
import os
from datasets import load_dataset
import helper_utils as utils
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
import os
from PIL import Image
import json
from datasets import load_dataset
from ultralytics import YOLO

print(tf.__version__)

/home/omar/TUM/05_projects/2D-Object-Detection/objDet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-23 21:09:23.472461: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-23 21:09:23.510145: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 21:09:23.510193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been register

2.15.0


# 0 Data preperation

In [2]:
# Create directories for saving images and annotations
image_dir = "./football_dataset/test/images"
annotations_dir = "./football_dataset/test/annotations"
if not os.path.exists(image_dir):  #check if the directory already exists
    ds = load_dataset("keremberke/football-object-detection")

    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(annotations_dir, exist_ok=True)

    # Save the dataset locally
    for i, item in enumerate(ds['test']):
        # Save image (directly save the PIL image)
        image = item['image']  # 'image' is already a PIL.JpegImageFile object
        image_path = os.path.join(image_dir, f"{i}.png")
        image.save(image_path)

        # Save annotations (e.g., bounding boxes)
        annotations = {
            "bboxes": item['objects']['bbox'],
        "object_id": item['objects']["id"]
    }
    annotations_path = os.path.join(annotations_dir, f"{i}.json")
    with open(annotations_path, 'w') as f:
        json.dump(annotations, f)

    print(f"Saved {len(ds['train'])} images and annotations locally.")


In [3]:
# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    device = "/GPU:0"
    print("GPU is available and will be used.")
else:
    device = "/CPU:0"
    print("GPU is not available. Using CPU.")

GPU is available and will be used.


2024-09-23 21:09:24.873387: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-23 21:09:24.910908: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-23 21:09:24.911113: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [12]:
import os
import json

def convert_to_yolo_format(annotations_dir, output_dir, image_shape):
    for json_file in os.listdir(annotations_dir):
        if json_file.endswith('.json'):
            with open(os.path.join(annotations_dir, json_file)) as f:
                annotations = json.load(f)

            image_w, image_h = image_shape
            txt_filename = os.path.splitext(json_file)[0] + '.txt'
            with open(os.path.join(output_dir, txt_filename), 'w') as f:
                for bbox in annotations['bboxes']:
                    x_min, y_min, width, height = bbox

                    # Normalize the bounding box coordinates
                    x_center = (x_min + width / 2) / image_w
                    y_center = (y_min + height / 2) / image_h
                    w = width / image_w
                    h = height / image_h

                    # Write the class and normalized bounding box
                    f.write(f"0 {x_center} {y_center} {w} {h}\n")
image_dir = "./football_dataset/test/images"
annotations_dir = "./football_dataset/val/annotations"
convert_to_yolo_format(annotations_dir=annotations_dir,output_dir = "./yolo_football_dataset/val/labels",image_shape=(1280,720))


In [15]:
from ultralytics import YOLO

# Load a YOLOv8 model pre-trained on COCO
model = YOLO('yolov8n.pt')  # 'n' stands for nano, you can also use 's', 'm', 'l', etc.

# Train the model on your dataset
model.train(
    data="data.yaml",  # Define the dataset configuration in a YAML file
    epochs=1,
    imgsz=720,  # Adjust this to match your image size
    batch=4,  # Adjust based on your GPU capacity
    name='football_yolov8'  # Name the training run
)
metrics = model.val()
print(metrics)


Ultralytics YOLOv8.2.99 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 3905MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=1, time=None, patience=100, batch=4, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=football_yolov83, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning /home/omar/TUM/05_projects/2D-Object-Detection/yolo_football_dataset/train/labels... 859 images, 1 backgrounds, 0 corrupt: 100%|██████████| 859/859 [00:00<00:00, 1219.21it/s]


train: New cache created: /home/omar/TUM/05_projects/2D-Object-Detection/yolo_football_dataset/train/labels.cache


val: Scanning /home/omar/TUM/05_projects/2D-Object-Detection/yolo_football_dataset/val/labels... 243 images, 0 backgrounds, 0 corrupt: 100%|██████████| 243/243 [00:00<00:00, 536.63it/s]

val: New cache created: /home/omar/TUM/05_projects/2D-Object-Detection/yolo_football_dataset/val/labels.cache


...

                   all        243       4513      0.924      0.844      0.891      0.552

1 epochs completed in 0.010 hours.
Optimizer stripped from /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov83/weights/last.pt, 6.2MB
Optimizer stripped from /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov83/weights/best.pt, 6.2MB

Validating /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov83/weights/best.pt...
Ultralytics YOLOv8.2.99 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 3905MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:03<00:00,  8.14it/s]


                   all        243       4513      0.924      0.844      0.891      0.552
       football_player        243       4513      0.924      0.844      0.891      0.552
Speed: 0.4ms preprocess, 3.5ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov83
Ultralytics YOLOv8.2.99 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 3905MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/omar/TUM/05_projects/2D-Object-Detection/yolo_football_dataset/val/labels.cache... 243 images, 0 backgrounds, 0 corrupt: 100%|██████████| 243/243 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:05<00:00, 11.88it/s]


                   all        243       4513      0.925      0.843      0.892      0.556
       football_player        243       4513      0.925      0.843      0.892      0.556
Speed: 0.5ms preprocess, 7.3ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov832
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x72507582b0a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022

In [16]:
metrics = model.val()
print(metrics)


Ultralytics YOLOv8.2.99 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 3905MiB)


val: Scanning /home/omar/TUM/05_projects/2D-Object-Detection/yolo_football_dataset/val/labels.cache... 243 images, 0 backgrounds, 0 corrupt: 100%|██████████| 243/243 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:03<00:00, 17.00it/s]


                   all        243       4513      0.925      0.843      0.892      0.556
       football_player        243       4513      0.925      0.843      0.892      0.556
Speed: 0.5ms preprocess, 6.4ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov833
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x725075a490c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022

In [18]:
results = model('football_dataset/test/images/4.png')

# Access the first result in the list
result = results[0]

# Display the results
result.show()
result.plot()  # This will plot the results on the image



image 1/1 /home/omar/TUM/05_projects/2D-Object-Detection/football_dataset/test/images/4.png: 416x736 16 football_players, 6.8ms
Speed: 7.9ms preprocess, 6.8ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 736)


array([[[  0,   6,  18],
        [212, 221, 234],
        [242, 246, 255],
        ...,
        [ 55,  23,  82],
        [ 44,  11,  72],
        [ 37,   4,  65]],

       [[246, 252, 255],
        [  8,  12,  23],
        [  5,   6,  16],
        ...,
        [ 84,  52, 111],
        [ 64,  31,  92],
        [ 50,  17,  78]],

       [[255, 252, 255],
        [  0,   0,   6],
        [ 34,  27,  34],
        ...,
        [130,  97, 154],
        [105,  71, 131],
        [ 87,  53, 113]],

       ...,

       [[  0,  24,   4],
        [ 87, 124, 104],
        [ 80, 124, 101],
        ...,
        [ 95, 148, 128],
        [ 75, 128, 108],
        [ 41,  94,  74]],

       [[  0,  27,   7],
        [ 83, 120, 100],
        [ 70, 114,  91],
        ...,
        [ 80, 133, 113],
        [ 60, 113,  93],
        [ 27,  80,  60]],

       [[  0,  10,   0],
        [ 58,  95,  75],
        [ 39,  83,  60],
        ...,
        [ 38,  91,  71],
        [ 18,  71,  51],
        [  0,  38,  18]]


(eog:70434): EOG-CRITICAL **: 21:09:25.361: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:70434): GLib-GIO-CRITICAL **: 21:09:25.361: g_file_equal: assertion 'G_IS_FILE (file1)' failed

(eog:70434): EOG-CRITICAL **: 21:09:25.567: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:70434): GLib-GIO-CRITICAL **: 21:09:25.567: g_file_equal: assertion 'G_IS_FILE (file1)' failed
